# 01. DQN with TicTacToe

Based on
https://github.com/lumiknit/TicTacToe-RL

오목 RL을 구현하기 전 틱택토를 먼저 다뤄봅니다.

## 틱택토 구현

우선 아래와 같이 TicTacToe 게임을 구현합니다.

(lumiknit/TicTacToe-RL/ttt.py)

아래 `Game`에 대해서 반드시 알아야 하는 것은 다음과 같습니다:

- 초기화 시 1차원 배열의 `board`를 생성.
- 0부터 8까지의 정수가 좌표가 됨. 각각 1행 1열, 1행 2열, ..., 3행 3열 순으로 할당.
- `.place(idx)`로 돌을 놓는 것이 가능.
- `.check_win()`으로 승패를 판단. 만약 둘 중 이긴 사람이 있으면 `1` 또는 `2` 반환, 무승부일경우 `0` 반환, 아직 승부가 안 났으면 `None` 반환.
- `.play(agent1, agent2)`를 통해 게임을 실행 가능. 이 `agent`는 `Game`의 instance를 인수로 받아서 다음 돌이 놓일 좌표를 반환하는 함수면 충분. (`user_input` 참고.)
- `.to_numpy`를 통해 `numpy.array`로 변환 가능.

In [1]:
import numpy as np

class Game:
  def __init__(self):
    self.board = [0] * 9
    self.turn = 1
    self.turns = 0
    self.foul = False

  def clone(self):
    g = Game()
    for i in range(9):
      g.board[i] = self.board[i]
    g.turn = self.turn
    g.turns = self.turns
    return g

  def rotate(self):
    # Clone and rotate clockwise
    g = Game()
    for r in range(3):
      for c in range(3):
        g.board[3 * r + c] = self.board[3 * (2 - c) + r]
    g.turn = self.turn
    g.turns = self.turns
    return g

  def place(self, idx):
    # Place stone at board[idx]
    # Iff the player cannot place a stone at idx, return False
    if self.board[idx] != 0:
      self.foul = True
      return False
    self.board[idx] = self.turn
    self.turn = 3 - self.turn
    self.turns += 1
    return True

  def check(self, i0, i1, i2):
    # Check whether 3 stones of a same color are placed in i0, i1, i2
    v = self.board[i0]
    if 0 != v and v == self.board[i1] and v == self.board[i2]:
      return v
    else: return None

  def check_win(self):
    # If some player 1 or 2 win, return 1 or 2
    # If they draw, return 0
    # Otherwise return None
    if self.foul:
      return 3 - self.turn
    r = None
    r = r or self.check(0, 1, 2) or self.check(3, 4, 5) or self.check(6, 7, 8)
    r = r or self.check(0, 3, 6) or self.check(1, 4, 7) or self.check(2, 5, 8)
    r = r or self.check(0, 4, 8) or self.check(2, 4, 6)
    if self.turns >= 9: r = r or 0
    return r
  
  def to_numpy(self, player):
    # Create 3 * 3 * 3 np array
    a = []
    for i in range(3):
      p = i
      if player == 2: p = (3 - p) % 3
      b = []
      for r in range(3):
        col = []
        for c in range(3):
          col.append(1. if self.board[r * 3 + c] == p else 0.)
        b.append(col)
      a.append(b)
    return np.array(a)

  def __str__(self):
    ch_arr = ['.', 'O', 'X']
    s = "---\nTurn: {} ({})\n".format(self.turn, ch_arr[self.turn])
    s += " | 0 1 2\n-+------"
    for r in range(3):
      s += "\n{}| ".format(r * 3)
      for c in range(3):
        s += "{} ".format(ch_arr[self.board[r * 3 + c]])
    return s

  def play(self, f1, f2, no_print=False):
    if no_print: p = lambda x: x
    else: p = print
    # Play game using two input functions
    if f1 == None: f1 = user_input
    if f2 == None: f2 = user_input
    tfn = [None, f1, f2]
    while self.turns < 9: # While empty cell exists
      p(str(self))
      # Take an index of cell to put a stone
      x = tfn[self.turn](self)
      p("{}P INPUT = {}".format(self.turn, x))
      # Check foul move.
      if type(x) != int or x < 0 or x >= 9 or self.board[x] != 0:
        p(str(self))
        p("Player {} put a stone on a non-empty cell!, Player {} win!" \
            .format(self.turn, 3 - self.turn))
        return
      # Place a stone
      self.place(x)
      # Check game is done
      w = self.check_win()
      if w == 0: # Draw
        p(str(self))
        p("Draw")
        return
      elif w != None: # Player `w` win
        p(str(self))
        p("Player {} win!".format(w))
        return
    p(str(self))
    p("Draw")

def user_input(game):
  i = None
  while i == None:
    try:
      # Read a index of cell (0~8)
      x = input("Idx(0~8) > ")
      i = int(x)
    except:
      print("Wrong Input!")
  return i

만약 직접 플레이를 해보고 싶다면 아래를 실행해보세요!

In [2]:
# Game().play() # Uncomment THIS!

틱택토의 경우에는 사람들이 이미 잘 알고 있는 전략이 존재합니다!
예를 들어서 아래와 같이 중앙/모서리를 공략하게 되면 비교적 손쉽게 무승부로 유도하는 것이 가능합니다.

(lumiknit/TicTacToe-RL/ttt_alg.py)

In [3]:
import random

def ttt_alg(game):
  # Simple strategy to win TicTacToe
  def c(i):
    # Check there are 2 same-color stones in a row
    v = max([game.board[x] for x in i])
    if v == 0: return None
    cnt = 0
    e = None
    for j in range(3):
      if v == game.board[i[j]]: cnt += 1
      else: e = i[j]
    if cnt != 2 or game.board[e] != 0: return None
    return e
  # The 1st or 2nd stone must be at the center or a corner
  if game.turns <= 1:
    if game.board[4] == 0: return 4
    else: return [0, 2, 6, 8][random.randrange(4)]
  # Otherwise, find there are 2 stones in a row
  r = c([0, 1, 2]) or c([3, 4, 5]) or c([6, 7, 8])
  r = r or c([0, 3, 6]) or c([1, 4, 7]) or c([2, 5, 8])
  r = r or c([0, 4, 8]) or c([2, 4, 6])
  if r != None: return r
  # Otherwise, just pick a random position
  for x in [4, 0, 2, 8, 6, 1, 7, 3, 5]:
    if game.board[x] == 0: return x

이 알고리즘을 이용하면 다음과 같은 플레이를 볼 수 있습니다.
(보통 사람이 게임을 하는 것 같지 않나요?)

In [4]:
Game().play(ttt_alg, ttt_alg)

---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . . 
1P INPUT = 4
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . . 
2P INPUT = 2
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . X 
3| . O . 
6| . . . 
1P INPUT = 0
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 
3| . O . 
6| . . . 
2P INPUT = 8
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O . X 
3| . O . 
6| . . X 
1P INPUT = 5
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 
3| . O O 
6| . . X 
2P INPUT = 3
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O . X 
3| X O O 
6| . . X 
1P INPUT = 6
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 
3| X O O 
6| O . X 
2P INPUT = 1
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O X X 
3| X O O 
6| O . X 
1P INPUT = 7
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O X X 
3| X O O 
6| O O X 
Draw


아래 코드를 실행해서 직접 알고리즘과 대결해보세요!

In [5]:
# Game().play(ttt_alg) # Uncomment THIS!

## Design Q function for TicTacToe

Q function은 현재 상태와 행동을 입력으로 받아서, 미래에 받을 잠재적인 보상을 출력하는 함수입니다.
여기서 상태와 행동은 게임에서 일어날 수 있는 모든 조합은 수용할 수 있어야 하며,
보상은 실제 보상이라기보다는,
다른 선택과 비교할 수 있는 상대적인 지표라고 생각하는 편이
낫습니다.

예를 들어서 TicTacToe의 경우에는

- 상태는 보드판이 됩니다. $3 \times 3$ 크기의 보드판에, 각 칸이 빈칸이거나 검정색/흰색 돌이 놓일 수 있으므로, 최대 $3^{9} = 19683$개의 경우의 수가 있습니다.
다만 실제로는 두 사람이 번갈아가면서 두기 때문에,
게임 중 나오는 경우는 이보다 적습니다.
- 행동은 돌을 각 보드판 어디에 두느냐가 됩니다.
판의 크기가 $9$이므로
행동의 경우의 수도 최대 $9$가 됩니다.
- TicTacToe에서 명확하고 객관적으로 부여할 수 있는
보상은 승패여부입니다. 예를 들어서 이기면 $r$,
지면 $-r$만큼 보상을 부여하고, 이 보상을 최대화 시키면
된다는 것입니다.

따라서 여기서의 Q function은
$$Q : \textbf{3}^{9} \times \textbf{9} \to \mathbb{R}$$
가 되겠네요.

다만, 실제로 Q function을 구현할 때에
행동을 입력으로 주기보다는,
각 행동별 보상을 모두 출력에 포함시키는 것이 편해서,
다음과 같이 만들게 됩니다.

$$Q : \textbf{3}^{9} \to \mathbb{R}^9$$

만약 이 $Q$의 출력의 최대값이 되는 위치가,
해당 상태에서 둘 수 있는 최선의 행동이라고 하면,
실제 agent는 $Q$를 평가하여 어디가 최선인지 알아내면 충분합니다.

### Q-Table (Only checking feasibilty)

만약 위 $Q$를 표로 구현한다고 해봅시다.
그러면 `Q`에는 입력과 출력 개수의 곱인
$3^{11} = 177147$개의 실수를 저장해야 합니다.
Single precision float로 저장한다면
692 kB 정도로 그다지 크지 않은 (사실 매우 작은)
메모리가 필요합니다.

이제 TicTacToe를 학습시킨다는 것이 `Q`라는 표에 값을 채워넣는 것으로 바뀌었습니다!

그런데 TicTacToe는 2인 제로섬 유한 완전정보 게임이다보니, 한쪽에 필승법이 존재하며,
이 때문에 저희는 `n`개 초과의 돌이 판에 놓인 경우
승패를 알 수 있다면, 현재 `n`개의 돌이 놓은 상태에서
아래와 같이 승부를 판단해볼 수 있습니다.

- `Q` 자체는 해당 수를 두었을 때 최종 결과를 나타냅니다.
- 만약 자신이 새로운 돌을 둔 상황을 `S'`이라고 할 때,
`Q(S')`의 값 중 '승'이 있다면,
상대방은 그 위치에 돌을 둠으로써 상대방이 이길 수 있습니다.
- 반대로 `Q(S')`이 모두 '패'라면
상대는 어디에 돌을 두든 상대방은 패할 수 밖에 없습니다.
- 즉 현재 상태 `S`가 있을 때, `Q(S)`는
`Q(S')` 중 '패'가 존재하면 '승', 그렇지 않다면 '패'라고 하면 문제가 없습니다.
- 편의상 `Q`는 현재 흑 차례일 때, 승률이라고 합시다. 만약 백이 어떤 수를 두어야 할지 판단해야 한다면, 판 위의 모든 돌의 색을 뒤바꾸면 됩니다.

위를 바탕으로 간단한 알고리즘을 작성해볼 수 있습니다.

```
let Q be table

; Fill every finish state
for each board S
  if S has 3-in-row
    Q[S][0..8] = 'WIN' if S has my 3-in-row else 'LOSE'
  elseif there is no empty cell in S
    Q[S][0..8] = 'WIN' ; Let's consider draw is better than lose...

; Fill other states
until Q is fully filed
  for each board S
    for each x in 0..8
      if a stone was placed at x in S
        Q[S][x] = 'LOSE' ; foul
      let S' be a state after I place a stone at x on S
      let S'' be S' flipped colors of every stone
      if some of Q[S''] is unfilled
        continue
      if Q[S''] contains 'WIN'
        Q[S][x] = 'LOSE'
      else
        Q[S][x] = 'WIN'
```

정말 다행인 점은 TicTacToe의 상태는 strictly increasing하다는 것입니다.
즉, 돌의 개수가 항상 증가하므로,
현재 상태는 다음 상태보다 돌의 개수가 적으며,
다음 상태의 `Q`가 모두 결정되면 (채워져 있으면)
현재 상태 역시 `Q`가 결정되므로,
무조건 종료를 하게 됩니다.

더 정확히는 맨 처음 반복문에서
돌이 9개 놓인 판의 `Q`가 모두 결정이 되며,
아래의 `until` 내부의 반복문은
현재까지의 `Q`에 대해 추가로 결정할 수 있는 판을 모두 찾아
`Q`를 갱신하므로,
적어도 처음에는 8개 돌이 놓인 판의 값을 모두 결정하고,
그 다음에는 7개 돌이 놓인 판의 값을 모두 결정하고, ...를
반복하게 됩니다.
때문에 모든 판의 경우의 수를 많아봐야 10번 훑으면
위 프로그램은 종료되며,
이 횟수는 1771470번으로, 길어봐야 10초면 해결이 됩니다.

## Q-Learning (Only checking feasibility)

위 방법은 TicTacToe에 필승법이 존재하기 때문에
가능합니다.
만약에 상태가 strictly increasing이 아니어서 몇 번의 행동으로 원래 상태로 돌아온다면 적용할 수 없습니다.
추가적으로 단순히 승/패가 아닌 복잡한 보상이 주어진다면
테이블을 채울 다른 방법이 필요하게 됩니다.

실제로는 정책 $\pi$에 대해
벨만 방정식과 Q-function은
$$
v_\pi(s) = \mathbb{E}_\pi(R_{t+1} + \gamma v_\pi(S_{t+1}) \mid S_t = s)
$$
$$
q_\pi(s, a) = \mathbb{E}_\pi(R_{t+1} + \gamma q_\pi(S_{t+1}, A_{t+1}) \mid S_t = s, A_t = a)
$$
와 같이 나타나며, 이 중 optimial Q-function은
$$
q_*(s, a) = \mathbb{E}(R_{t+1} + \gamma \max_{a' \in \mathcal{A}} q_*(S_{t+1}, a') \mid S_t=s, A_t = a)
$$

Q-Learning을 하게 되면
다음과 같은 식을 통해 수렴을 시키게 됩니다:

$$
Q(s, a) \leftarrow (1 - \alpha) Q(s, a) + \alpha( R(s, a) + \gamma \max_{a' \in A} Q(s', a'))
$$

위의 테이블의 값을 갱신하는 부분은
여기서 보상인 $R$을 승/패에 따라 적당한 두 실수로 둔 것이고,
discount factor $\gamma$를 1로 두었다고 생각하면 됩니다.

## Deep Q Network

Q-function을 일일이 학습을 시키려면
저 표를 모두 관리하면서 조금씩 수렴을 시켜야 하는데,
단순하게 봐도 심각한 문제를 찾을 수 있습니다.

- 표가 너무 큽니다. 예를 들어서 오목은 상태만 $3^{15\times15} \simeq 2^{356}$개가 되는데, 이 상태 개수만큼 실수를 저장하는 것도 불가능합니다.
- 한번에 상태 및 행동 조합 중 한가지에 대해서만 갱신을 합니다. 위의 경우의 수와 맞물려서 학습이 언제 끝날지 감도 안 오게 만드는 주 원인입니다.

그래서 표 대신에 샘플을 바탕으로 함수를 근사하는
알고리즘을 생각하는 것이고, 여기에 deep learning을
적용해보자는 것이 DQN의 기본적인 아이디어입니다.

실제로 DQN을 학습시키기 위해서는 세세한 조정이 필요하지만,
여기서는 간단하게 구현해보도록 합니다.

(lumiknit/TicTacToe-RL/agent.py)

우선 Torch를 불러옵니다.

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

학습 전, `Game`을 `tensor`로 바꾸는 부분과
랜덤 행동을 발생시키는 함수,
돌을 둘 수 없는 곳의 칸을 $-\infty$ 로 설정하는 함수를 만듭니다.

In [7]:
def game_to_input(game):
  n = game.to_numpy(game.turn)
  r = np.random.rand(3, 3, 3) / 10.0
  return torch.tensor((n + r), dtype=torch.double) \
      .reshape(3 * 3 * 3)

def pick_random_move(game):
  t = None
  v = np.random.rand()
  idx = np.random.randint(9)
  for off in range(9):
    rdx = (idx + off) % 9
    if game.board[rdx] == 0:
      t = rdx
      g = game.clone()
      g.place(rdx)
      if g.check_win() == None: break
  return t

def cut_foul(game, qvs):
  q = qvs.clone().detach()
  for i in range(9):
    if game.board[i] != 0:
      q[i] = -100.0
  return q

이제 모델을 만듭니다.

보드게임을 학습할 때는 CNN을 주로 사용한다는 것 같지만,
TicTacToe는 크기가 매우 작으므로 그냥 dense network를 구성합니다.

In [25]:
model = nn.Sequential(
    nn.Linear(3 * 3 * 3, 243).double(),
    nn.ReLU(),
    nn.Linear(243, 36).double(),
    nn.ReLU(),
    nn.Linear(36, 36).double(),
    nn.ReLU(),
    nn.Linear(36, 3 * 3).double(),
)

loss_fn = nn.MSELoss()

gamma = 0.95

다음에는 돌을 두었을 때 reward를 포함해서
새롭게 갱신될 Q-function의 값을 계산하는 함수를 만듭니다.

In [9]:
def calc_Q(game, idx):
  # Calculate reward obtained when a stone is put on idx cell
  if not game.place(idx): # Foul: Q = -1
    return -1.0, game.turn
  else:
    w = game.check_win()
    if w == None: # Not finished: Q = - max_a Q(s', a)
      with torch.no_grad():
        qvs2 = cut_foul(game, model(game_to_input(game)))
      return (0 - gamma * torch.max(qvs2).item()), w
    elif w == 0: # Draw: Q = 0
      return 0.0, w
    elif w == 3 - game.turn: # Win: Q = 1
      return 1.0, w
    else: # Lose: Q = -1
      return -1.0, w

이제 본격적으로 학습을 시킵니다.

In [24]:
def learn():
  # Deep learning on Q-NN

  learning_rate = 0.02

  # I don't know why Adam optimizer does not work well
  # optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
  # Use SGD+Momentum optimizer
  optimizer = optim.SGD(
      model.parameters(),
      lr=learning_rate,
      momentum=0.9,
      weight_decay=1e-5)

  # Exploration parameter
  epsilon = 0.25

  # # of epochs
  n_epoch = 20000
  # Epoch when epsilon start to decrease
  dec_ep_epoch = 3000
  # Epsilon decreasement factor
  ep_dim = 0.9999
  # Lower-bound of epsilon
  ep_lb = 0.05
  # Interval of printing learning state
  print_interval = 200

  # Loss statistics
  n_loss = 0
  acc_loss = 0

  # Start running
  for epoch in range(n_epoch):
    # Make a new game board
    game = Game()
    while True:
      # Calculate good answer by my strategy
      alg_idx = ttt_alg(game)
      # Calculate Q-val
      qvs = model(game_to_input(game))
      # Make an expected Q vector
      Y = qvs.clone().detach()
      # Explore all cells
      for idx in range(9):
        # Put a stone if we can
        if game.board[idx] != 0: continue
        g = game.clone()
        # State transition & calculate reward
        nq, w = calc_Q(g, idx)
        # Put a reward into an expected Q vector
        Y[idx] = nq
      # Back propagation
      optimizer.zero_grad()
      loss = loss_fn(qvs, Y)
      loss.backward()
      optimizer.step()
      # Update loss
      acc_loss += loss.item()
      n_loss += 1
      # Put a stone
      # If rand() < epsilon, go to random
      # if rand() < 2 epsilon, use my strategy
      # otherwise, use Q-NN
      idx = None
      with torch.no_grad():
        qvs = cut_foul(game, model(game_to_input(game)))
      rnd = np.random.rand()
      if rnd < epsilon:
        idx = pick_random_move(game)
      elif rnd < 2 * epsilon:
        idx = ttt_alg(game)
      else:
        idx = torch.argmax(qvs).item()
      # Put a stone and check the game is done
      nq, w = calc_Q(game, idx)
      if w != None:
        break
    # Print loss statistics
    if epoch % print_interval == print_interval - 1:
      print("{:6d}: e={:.4f}; L = {:8.4f}" \
          .format(epoch + 1, epsilon, acc_loss / n_loss))
      acc_loss = 0
      n_loss = 0
    # Decrease epsilon
    if epoch > dec_ep_epoch:
      epsilon *= ep_dim
      if epsilon < ep_lb: epsilon = ep_lb


In [26]:
learn()

 18000: e=0.0558; L =   0.0190
 18200: e=0.0547; L =   0.0161
 18400: e=0.0536; L =   0.0108
 18600: e=0.0525; L =   0.0093
 18800: e=0.0515; L =   0.0098
 19000: e=0.0505; L =   0.0115
 19200: e=0.0500; L =   0.0098
 19400: e=0.0500; L =   0.0120
 19600: e=0.0500; L =   0.0102
 19800: e=0.0500; L =   0.0106
 20000: e=0.0500; L =   0.0140


이 모델을 바탕으로 행동하는 agent는 다음과 같습니다.

In [27]:
def model_action(game):
  global model
  with torch.no_grad():
    qv = cut_foul(game, model(game_to_input(game)))
  return torch.argmax(qv).item()

이를 바탕으로 실제 게임을 돌려보도록 하죠.

In [28]:
Game().play(model_action, ttt_alg)

---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . . 
1P INPUT = 8
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . O 
2P INPUT = 4
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . X . 
6| . . O 
1P INPUT = 1
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . O . 
3| . X . 
6| . . O 
2P INPUT = 0
---
Turn: 1 (O)
 | 0 1 2
-+------
0| X O . 
3| . X . 
6| . . O 
1P INPUT = 2
---
Turn: 2 (X)
 | 0 1 2
-+------
0| X O O 
3| . X . 
6| . . O 
2P INPUT = 5
---
Turn: 1 (O)
 | 0 1 2
-+------
0| X O O 
3| . X X 
6| . . O 
1P INPUT = 3
---
Turn: 2 (X)
 | 0 1 2
-+------
0| X O O 
3| O X X 
6| . . O 
2P INPUT = 6
---
Turn: 1 (O)
 | 0 1 2
-+------
0| X O O 
3| O X X 
6| X . O 
1P INPUT = 7
---
Turn: 2 (X)
 | 0 1 2
-+------
0| X O O 
3| O X X 
6| X O O 
Draw


이제 이를 바탕으로 승률을 확인해봅시다.
먼저 랜덤으로 돌을 두는 agent는 다음과 같습니다.

In [29]:
def rand_action(game):
  # Play TTT randomly
  idx = np.random.randint(9)
  for off in range(9):
    rdx = (idx + off) % 9
    if game.board[rdx] == 0: return rdx
  return idx

여기에 승률 측정용 함수,

In [30]:
def run_games(a, b, rounds):
  a_win = 0
  a_draw = 0
  a_lose = 0
  for i in range(rounds):
    g = Game()
    sw = False
    if np.random.rand() < 0.5:
      sw = False
      g.play(a, b, True)
    else:
      sw = True
      g.play(b, a, True)
    w = g.check_win()
    if w != None and w >= 1 and sw: w = 3 - w
    if w == 1: a_win += 1
    elif w == 2: a_lose += 1
    else: a_draw += 1
  print("RESULT: {} win / {} draw / {} lose // {} rounds"
    .format(a_win, a_draw, a_lose, rounds))

In [31]:
print("-- v.s. Random")
run_games(model_action, rand_action, 1000)

-- v.s. Random
RESULT: 889 win / 82 draw / 29 lose // 1000 rounds


In [33]:
print("-- v.s. Algorithm")
run_games(model_action, ttt_alg, 1000)

-- v.s. Algorithm
RESULT: 0 win / 1000 draw / 0 lose // 1000 rounds


Nueral network에서 계산이 오래 걸리기는 하지만,
약 20,000번의 게임을 진행하여 학습을 하였고,
알고리즘의 경우 거의 모든 경우에 질 수 없는 방식이기 때문에
알고리즘과는 호각으로 싸우며,
랜덤에 대해서는 압도적으로 이기는 것을 볼 수 있습니다.
다만, 모든 수에 대해 학습이 되지는 않기 때문에
일부 패배하는 경우가 있음을 알 수 있습니다.